In [0]:
#@title Config
_git_branch = "contracts-subj" #@param {type:"string"}
GLOBALS__={}

In [0]:
#@title Select sections to run: { run: "auto", vertical-output: true, display-mode: "form" }

#@markdown 1. ### Charters
batch_charters = False #@param {type:"boolean"}
batch_charters_contents = False #@param {type:"boolean"}

#@markdown 2. ### Protocols
batch_protocols = False #@param {type:"boolean"}

#@markdown 3. ### Contracts

batch_contract_contents = False #@param {type:"boolean"}
batch_contract_find_sections = False #@param {type:"boolean"}
batch_contracts = True #@param {type:"boolean"}

In [0]:
''' AZ:-IMPORT CODE GITHUB----------------------------------------------😺------ '''
import sys


def _init_import_code_from_gh():
  if 'GLOBALS__' not in globals():
    print('adding global GLOBALS__')
    global GLOBALS__
    GLOBALS__ = {}

  if '_init_import_code_from_gh' in GLOBALS__:
    print('👌 code already imported from GitHub!')
    return

  import subprocess
  def exec(x):
    r = subprocess.check_output(x, shell=True)
    r = r.decode('unicode-escape').encode('latin1').decode('utf8')
    print(r)

  print(f"fetching code from GitHub.....{_git_branch}")
  try:
    exec('rm -r nlp_tools')
  except:
    pass
  exec(f'git clone --single-branch --branch {_git_branch} https://github.com/compartia/nlp_tools.git nlp_tools')

  print('🦊 GIT revision:')
  exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

  sys.path.insert(0, 'nlp_tools')

  # self-test
  from text_tools import untokenize
  print(untokenize(['code', 'imported', 'OK 👍']))

  print('installing antiword...')
  exec('sudo apt-get install antiword')

  print('installing docx2txt...')
  exec("pip install docx2txt")

  GLOBALS__['_init_import_code_from_gh'] = True

  print('❤️ DONE importing Code fro GitHub')

''' AZ:-------------------------------------------------IMPORT CODE GITHUB-😺---'''


In [0]:
 

# AZ:-INIT ELMO-----------------------------------------------------------------------------------

import tensorflow as tf
import tensorflow_hub as hub



def _import_elmo():
  """
  ACHTUNG!! this method is called later by ElmoEmbedder
  """
  
#   if 'elmo' in GLOBALS__:
#     print('👌 Tensorflow hub.Module is already imported ')
#     return GLOBALS__['elmo']

  
  elmo = hub.Module('https://storage.googleapis.com/az-nlp/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz',
                    trainable=False)  # twitter
  print('❤️ ❤️ ❤️ DONE (re)importing Tensorflow hub.Module ')
  print('Tensorflow version is', tf.__version__)

  return elmo
#   GLOBALS__['elmo'] = elmo

#   print(GLOBALS__['elmo'].__dict__)
#   return GLOBALS__['elmo']


# AZ:-INIT EMBEDDER-----------------------------------------------------------------------------------

def _init_embedder():
  if 'elmo_embedder' in GLOBALS__:
    print('👌 Embedder is already created! ')
    return


  from embedding_tools import ElmoEmbedder
  GLOBALS__['elmo_embedder'] = ElmoEmbedder(_import_elmo(), tf, 'elmo', _import_elmo)
  
  print('❤️ DONE creating words embedding model')
  return GLOBALS__['elmo_embedder']


# AZ:-Init chartes context-----------------------------------------------------------------------------------
def _init_charters():
  if 'CharterAnlysingContext' in GLOBALS__:
    print('👌 Charters-related tools are already inited ')
    return

  from charter_parser import CharterDocumentParser
  GLOBALS__['CharterAnlysingContext'] = CharterDocumentParser(None)
  print('❤️ DONE initing Charters-related tools and models ')


def _init_contracts():
  if 'ContractAnlysingContext' in GLOBALS__:
    print('👌 Contracts-related tools are already inited ')
    return

  from contract_parser import ContractAnlysingContext
  GLOBALS__['ContractAnlysingContext'] = ContractAnlysingContext(GLOBALS__['elmo_embedder'], GLOBALS__['renderer'])
  print('❤️ DONE initing Contracts-related tools and models ')


# AZ:- THE CODE----------------------------------------------------------------------------------
def _init_the_code():
  if '_init_the_code' in GLOBALS__:
    print('👌 Code is alredy imported!')
    return

  from renderer import SilentRenderer

  class RendererForBatch(SilentRenderer):
    pass

  GLOBALS__['renderer'] = RendererForBatch()
  GLOBALS__['_init_the_code'] = True

  def read_doc(fn):
    import docx2txt, sys, os
    text = ''
    try:
      text = docx2txt.process(fn)
    except:
      print("Unexpected error:", sys.exc_info())
      os.system('antiword -w 0 "' + fn + '" > "' + fn + '.txt"')
      with open(fn + '.txt') as f:
        text = f.read()

    return text

  GLOBALS__['read_doc'] = read_doc

  print("❤️ DONE initializing the code")

# AZ:---------------------------------------------------------------------------END OF THE THE CODE, See you later


In [0]:
## do preparation here   
    
#1.
_init_import_code_from_gh()
#2.
_init_embedder()
#3.
_init_the_code()
#4. 
if batch_charters:
  _init_charters()
if batch_contracts:
  _init_contracts()

# BATCH
пакетный процессинг уставов, запись результатов в google sheets

## Authenticate on Google and mount Google Drive

In [0]:
import gspread
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
google_spread = gspread.authorize(GoogleCredentials.get_application_default())

drive.mount('/content/gdrive', force_remount=True)



In [0]:
import glob
def read_documents(filename_prefix):
  texts = {}
  for file in glob.glob(filename_prefix+"*.doc"):
    try:
      text = GLOBALS__['read_doc'](file)
      texts[file] = text
      print("good:", file)
    except:
      print('WRONG *.doc FILE!!', file)

  for file in glob.glob(filename_prefix+"*.docx"):
    try:
      text = GLOBALS__['read_doc'](file)
      texts[file] = text
      print("good:", file)
    except:
      print('WRONG *.docx FILE!!', file)
      
  return texts

In [0]:
import sys
import traceback
import gc
    
#-------------------------------------------------------------------------------
  
import time
def run_batch(batch_method, worksheet, filename_prefix, documents) :
  


#   worksheet.update_cell(1, 1, "used CONFIG: {}:".format(str(hyperparameters)))
  
  
  # _row=3

  the_row = start_from_row
  number_of_files = int(worksheet.cell(1, 1).value)

  col = 1
  for _row in range(start_from_row, 2 + number_of_files):
    gc.collect()

    def _clean():
      _clean_row(worksheet, the_row, 3, 15)


    print("=" * 120)
    print("=" * 120)
    short_fn = worksheet.cell(_row, 1).value

    try:
      the_row = int(worksheet.cell(_row, 2).value)
    except:
      worksheet.update_cell(_row, 2, the_row)
      
    worksheet.update_cell(the_row, 5, '-PENDING...-')
      
    _clean_row(worksheet, the_row, 3, 12)
    _clean_row(worksheet, the_row+1, 3, 12)
    _clean_row(worksheet, the_row+2, 3, 12)
 

    filename = filename_prefix + short_fn
    print(_row, '-', the_row, filename)    

    
    worksheet.update_cell(the_row, col + 2, "{}: {}".format(_row, short_fn))
    worksheet.update_cell(the_row, col + 3, time.strftime("%Y-%m-%d %H:%M"))
    
    try:      
      #===========================================
      the_row = 1 + batch_method(documents, worksheet, filename, the_row, col+4)
      #===========================================
    except Exception as e:
      _, _, tb = sys.exc_info()
      traceback.print_tb(tb) # Fixed format
      tb_info = traceback.extract_tb(tb)
      filename, line, func, text = tb_info[-1]

      print("Unexpected error:", sys.exc_info())
      worksheet.update_cell(the_row, col + 11, f'-ERROR- {func} {line} {text} {sys.exc_info()}' )
      raise(e)

    the_row += 1
    worksheet.update_cell(_row + 1, 2, the_row)

# CHARTERS

In [0]:
if batch_charters: 
  from charter_patterns import CharterPatternFactory
  
  CPF = CharterPatternFactory( GLOBALS__[ 'elmo_embedder']  )
  print( [ p.name for p in  CPF.patterns]  )
  GLOBALS__['CharterAnlysingContext'].pattern_factory = CPF
  
  
  


In [0]:
###

### Microtest

In [0]:
microsample = """
Общие положения

1.1. В соответствии с настоящим Договором Жертвователь обязуется безвозмездно передать Получателю денежные средства в размере 30 000 (Тридцать тысяч) рублей в качестве пожертвования.

"""

if 'CharterAnlysingContext' in GLOBALS__:
  GLOBALS__['CharterAnlysingContext'].analyze_charter(microsample)
  print('ok')
else:
  print('🔞 CharterAnlysingContext is NOT initialized, call _init_charters() before')

## 1

In [0]:
# raise error

In [0]:
#@title Заполнение XLS таблицы { run: "auto", vertical-output: true, form-width: "650px", display-mode: "both" }

populate_names = False  #@param {type: "boolean"}
#@markdown - читает список файлов из папки и заносит их имена в первый столбец таблицы
start_from_row = 2  # @param {type: "integer"}
#@markdown - читает имена файлов из первого столбца начиная со строки  start_from_row (min=2)

if start_from_row < 2:
  raise Exception()

In [0]:
def _clean_row(worksheet, r, s, e, fill=''):
  for col in range(s, e + 1):
    worksheet.update_cell(r, col, fill)
  worksheet.update_cell(r, e, '-EOF-')

  

def _populate_org(orginfo, the_row, col) -> int:
  from text_tools import untokenize 
  worksheet.update_cell(the_row, col , orginfo['name'])
  worksheet.update_cell(the_row, col + 1, orginfo['type_name'])
#   worksheet.update_cell(the_row, col + 2, orginfo['type'])
  worksheet.update_cell(the_row, col + 2, untokenize(orginfo['tokens'])[0:300] )

  return the_row+1
  
  
 

from legal_docs import LegalDocument
 



def render_contents(ws, doc: LegalDocument, _row, _col):
  r = _row + 1
  col = _col
  ws.update_cell(r, col, 'Выявленное Содержание')

  for n in range(len(doc.structure.headline_indexes) - 1):
    i = doc.structure.headline_indexes[n]
    i_next = doc.structure.headline_indexes[n + 1]

    sline = doc.structure.structure[i]
    sline_next = doc.structure.structure[i_next]

    line = [
      '',#B erase
      '',#C
      '',#D
      str(i),#E
      sline.to_string(doc.tokens_cc),#F
      sline_next.span[0]-sline.span[1],
      sline.line_number,
      sline.minor_number,
      str(sline.number),
      sline.level,
      sline._possible_levels[0],
      sline.span[1]
      
    ]
    r+=1
    
    cell_list = ws.range(f'B{r}:N{r}')

    i=0
    for cell in cell_list:
      if i<len(line):
        cell.value = line[i]
      i+=1

    ws.update_cells(cell_list)
  
  return r

### Rread all chaters

In [0]:
if batch_charters or batch_charters_contents:

  charters_filename_prefix='/content/gdrive/My Drive/GazpromOil/Charters/'
  charters = read_documents(charters_filename_prefix)


##  Test population

In [0]:
TEST_POPULATION=False


if TEST_POPULATION:
  CTX=GLOBALS__['CharterAnlysingContext']
  print(CTX.__dict__)
  CTX.analyze_charter(charters[charters_filename_prefix + 'ГПН Устав.docx' ])
print('OK')

In [0]:
worksheet = google_spread.open('Charter test results').sheet1  

In [0]:
# CTX.doc
# CTX.org
# CTX.constraints

#-------------------------
#todo: make renderer
from parsing import head_types_dict


def _clean(ws, r):
    start='C'
    end = 'Z'
#     end = chr(ord(start) + len(line)-1)
    cell_list = ws.range(f'{start}{r}:{end}{r}')
    for cell in cell_list:
      cell.value = ''
    ws.update_cells(cell_list)


def _populate_rz(charter, r, col, ws):
  from structures import OrgStructuralLevel
  from patterns import PatternSearchResult
  from renderer import org_level_dict
  
  def _p(c, val):
    try:
      ws.update_cell(r, col + c, val)
    except:
      print(f'cannot update cell {r} {col+c} with value"{val}"')

      
      
  for level in OrgStructuralLevel:
    constraint_search_results: List[PatternSearchResult] = charter.constraints_by_org_level(level)
      
    _clean(ws, r)

    cell_list = ws.range(f'F{r}:H{r}')
    
    cell_list[0].value = org_level_dict[level].upper()
    cell_list[1].value = " headline: TODO:" #+ r_by_head_type['caption'][0:300]     
    ws.update_cells(cell_list)
     
    renderered=0
    for sentence in constraint_search_results:      
      r = populate_constraints(sentence, r+1, ws)
      renderered+=1
    if renderered==0:
      r+=1
      _clean(ws, r)
      ws.update_acell(f'G{r}', 'пустота пустоты НАЙДЕНА ॐ ओङ्कार 🐼 ом шанти шанти  ॐ' )
      
      r+=1
    _clean(ws, r)
    _clean(ws, r+1)
    _clean(ws, r+2)
    _clean(ws, r+3)

  return r+1


from legal_docs import LegalDocument

ggg={
    'RealEstate':'🐌 Неподвижность',
    'Charity':'🙏 Благ-ость',
    'Lawsuit':'🏦 Иски',
    'Other':'👽неизвестняк (other)'
}
  
def populate_constraints(sentence , row, ws):
  subj_type = sentence.subject_mapping["subj"]
  _clean(ws, row)
        
  constraints: List[ValueConstraint] = sentence.constraints
  sname = ggg [subj_type.name]
  
  ws.update_acell(f'G{row}', " ".join( sentence.tokens[0:200] ))
  ws.update_acell(f'F{row}', sname )

  
  if len(constraints) > 0:
    
    for pv in constraints: 
      row+=1
      _clean(ws, row)
      c = pv.value
      if c.value is not None and c.sign is not None:
        col= 10 - c.sign * 2
        ws.update_cell(row, col, c.value)       
        ws.update_cell(row, col+1, c.currency)      
  else:
    row+=1
    _clean(ws, row)

    ws.update_acell(f'G{row}', '- суммы не найдены -' )
    
  row+=1
  _clean(ws, row)
  return row 



if TEST_POPULATION:
  
#   _populate_org(CTX.org, 2, 2)  
  _populate_rz(CTX.charter, 2, 5, worksheet)

In [0]:

def charter_batch_method(documents, worksheet, filename, r, col) -> int:

  text = documents[filename]
  worksheet.update_acell(f'E{r}', 'Parsing 🧚, .... be patient 🙏 ' )
  GLOBALS__['CharterAnlysingContext'].analyze_charter(text)

  _clean_row(worksheet, r, col, 20)
  _clean_row(worksheet, r+1, col, 20)
  _clean_row(worksheet, r+2, col, 20)
  
  r = _populate_org(GLOBALS__['CharterAnlysingContext'].charter.org, r, col)    
  r = _populate_rz(GLOBALS__['CharterAnlysingContext'].charter, r, col+1, worksheet)
  
  return r + 1




def charter_c_batch_method(documents, worksheet, filename, r, col) -> int:

  text = documents[filename]
  doc = CharterDocument(text)

    # 1. find top level structure
  doc.parse()


  _clean_row(worksheet, r, col, 20)
  _clean_row(worksheet, r+1, col, 20)
  _clean_row(worksheet, r+2, col, 20)
  
  r=render_contents(worksheet, doc, r, col)
  del doc
  
  _clean_row(worksheet, r+1, col, 20)
  _clean_row(worksheet, r+2, col, 20)
  worksheet.update_cell(r+1, 4, 'END of DOC') 
  worksheet.update_cell(r+3, 4, 'END of ALL') 
  
  
  return r + 1


## 🤬 populate contents only

In [0]:
print(f'batch_charters_contents={batch_charters_contents}')
if batch_charters_contents:
  for k in charters:
    print(k)
  
  start_from_row=2
  worksheet = google_spread.open('Charter test results').worksheet('_contents')

  #TAKES TIME=====================================================================
  if batch_charters_contents:
    run_batch(charter_c_batch_method, worksheet, charters_filename_prefix, charters)
  #===============================================================================


## run batch loop

In [0]:
if batch_charters:
  worksheet = google_spread.open('Charter test results').sheet1  

if populate_names:
  import os, sys
  _row=2  

  # Populate document names
  for filename in sorted(charters):
    head, tail = os.path.split(filename)  
    worksheet.update_cell( _row, 1, tail)

    _row+=1

    
    
    
#TAKES TIME=====================================================================
if batch_charters:
  run_batch(charter_batch_method, worksheet, charters_filename_prefix, charters)
#===============================================================================

# CONTRACTS

#### Rread all Contracts

In [0]:

if batch_contracts or batch_contract_contents or batch_contract_find_sections:

  contracts_filename_prefix='/content/gdrive/My Drive/GazpromOil/Contracts/'
  contracts = read_documents(contracts_filename_prefix)


In [0]:
 
from text_tools import untokenize
from renderer import known_subjects_dict
from typing import List

from contract_parser import ContractDocument3
from ml_tools import ProbableValue
from renderer import known_subjects_dict
from structures import ContractSubject

def render_subj(contract: ContractDocument3):
  subjects: List[ProbableValue] = contract.subjects

  if len(subjects) > 0:
    sorted_ = [y for y in sorted(subjects, key=lambda x: -x.confidence)]
    subject_kind = sorted_[0].value
    confidence = sorted_[0].confidence
  else:
    subject_kind = ContractSubject.Other

  if subject_kind in known_subjects_dict:
    rendering_name = known_subjects_dict[subject_kind]
  else:
    rendering_name = 'прочее'

  return rendering_name, subject_kind, confidence
   
  
def contract_batch_method(docs, worksheet, filename, r, col) -> int:

  text = docs[filename]
  CTX = GLOBALS__['ContractAnlysingContext']
  doc, values = CTX.analyze_contract(text)

  _clean_row(worksheet, r, col, 20)
  _clean_row(worksheet, r+1, 3, 20)
  _clean_row(worksheet, r+2, 3, 20)
  _clean_row(worksheet, r+3, 3, 20)
  
  quote_col = col
  pricequote_col = quote_col+1
  
  if 'subj' in doc.sections:
    section = doc.sections['subj']    
    body = section.body.untokenize_cc()[:1000]
    headline = section.subdoc.untokenize_cc()[:500]
    
    worksheet.update_cell(r, quote_col, headline+"\n" +body)
  else:
    worksheet.update_cell(r, quote_col, '⚠️ раздел о предмете любви не обнаружен')

#   subj = doc.subject[0]
#   subj_c = doc.subject[1]
  
  rendering_name, subject_kind, confidence = render_subj(doc)
  
  worksheet.update_cell(r, pricequote_col+1, confidence)
  worksheet.update_cell(r, pricequote_col+2, rendering_name)
  
 
  
  q=''
  
  if len(values)>0:
    cnt = pricequote_col+2
    
    for pc in values:      
      #saving quotes
      c = pc.value
      q += untokenize(c.context.tokens)+'\n'
      q = q[0:500]
      
      

    for pc in values:      
      c = pc.value
      worksheet.update_cell(r+2, cnt+1, c.currency)
     
      worksheet.update_cell(r+1, cnt, c.sign)       
      worksheet.update_cell(r+1, cnt+1, c.value)  
      worksheet.update_cell(r+1, cnt+2, pc.confidence)  
      
      cnt+=3
  else:
    worksheet.update_cell(r+1, pricequote_col+1, "⚠️ ничегошеньки не найдено!")   
  
  q+='\n\n'+CTX.get_warings()
  worksheet.update_cell(r, pricequote_col, q)  
  
  return r + 3
      



### run batch loop

In [0]:
#@title Run Contracts batch { form-width: "300px" }

start_from_row=14 #@param {type:"integer"}
if batch_contracts:
  worksheet = google_spread.open('Charter test results').worksheet('Contracts')
import os
  
if populate_names or True:

  _row=2  

  # Populate document names
  for filename in sorted(contracts):
    head, tail = os.path.split(filename)  
    worksheet.update_cell( _row, 1, tail)

    _row+=1

if batch_contracts:
  run_batch(contract_batch_method, worksheet, contracts_filename_prefix, contracts)

## 🤬 populate Contract contents only

In [0]:
 
start_from_row=2 #@param {type:"integer"} 
import os

def contract_c_batch_method(docs, filename, r, col) -> int:

  text = docs[filename]
  doc = LegalDocument(text)
  doc.parse()

  _clean_row(worksheet, r, col, 20)
  _clean_row(worksheet, r+1, col, 20)
  _clean_row(worksheet, r+2, col, 20)
  
  r = render_contents(worksheet, doc, r, col)
  del doc
  
  _clean_row(worksheet, r+1, col, 20)
  _clean_row(worksheet, r+2, col, 20)
  worksheet.update_cell(r+1, 4, 'END of DOC') 
  worksheet.update_cell(r+3, 4, 'END of ALL') 
  
  return r + 1



print(f'batch_contract_contents={batch_contract_contents}')
if batch_contract_contents:
  

  worksheet = google_spread.open('Charter test results').worksheet('_contents_contracts')
  
  _row=2  
  # Populate document names
  for filename in sorted(contracts):
    head, tail = os.path.split(filename)  
    worksheet.update_cell( _row, 1, tail)
    _row+=1

  #----------
#   batch_contract_find_sections
  #----------
  

  #TAKES TIME=====================================================================
  if batch_contract_contents:
    run_batch(contract_c_batch_method, worksheet, contracts_filename_prefix, contracts)
  #===============================================================================


## 🤬 populate Contract Sections info 🏛

In [0]:
 
start_from_row=27 #@param {type:"integer"} 
import os




def populate_sections(ws, sections, _row, _col):
  r = _row + 1
  col = _col
  ws.update_cell(r, col, 'Выявленное секции')
  
  
  for section_type in sections:
    section:HeadlineMeta = sections[section_type]
    line=[
        section.index,
        section.confidence,
        section_type,        
        section.subdoc.untokenize_cc()[:500],        
        len(section.body.tokens),
        section.body.untokenize_cc()[:700]
        
    ]
    r+=1
    start='E'
    end = chr(ord(start) + len(line)-1)
    cell_list = ws.range(f'{start}{r}:{end}{r}')

    i=0
    for cell in cell_list:
      cell.value = line[i]
      i+=1

    ws.update_cells(cell_list)

  

  return r
  
  
  
def contract_sections_batch_method(docs, filename, r, col) -> int:

 
  CTX = GLOBALS__['ContractAnlysingContext']
  doc = LegalDocument( docs[filename] )
  doc.parse()
  CTX.contract = doc
  print("1000 TOKENS:",CTX.contract.untokenize()[0:1000])
  CTX.contract.embedd(CTX.hadlines_factory)
  CTX._logstep("parsing document and detecting document high-level structure")

  sections = CTX.sections_finder.find_sections(doc, 
                             CTX.hadlines_factory, 
                             CTX.hadlines_factory.headlines,
                             headline_patterns_prefix='headline.')

  

  _clean_row(worksheet, r, col, 20)
  _clean_row(worksheet, r+1, col, 20)
  _clean_row(worksheet, r+2, col, 20)
  
  r = populate_sections(worksheet, sections, r, col)
  CTX._reset_context()
  del doc
  
  _clean_row(worksheet, r+1, 3, 20)
  _clean_row(worksheet, r+2, 3, 20)
  _clean_row(worksheet, r+3, 3, 20)
  _clean_row(worksheet, r+4, 3, 20)
  worksheet.update_cell(r+2, 4, 'END of DOC') 
  worksheet.update_cell(r+3, 4, 'END of ALL') 
  
  return r + 1



print(f'batch_contract_find_sections={batch_contract_find_sections}')
if batch_contract_find_sections:
  _init_contracts()
  CTX = GLOBALS__['ContractAnlysingContext']
  from sections_finder import FocusingSectionsFinder
  assert CTX is not None
  
  sf = FocusingSectionsFinder(CTX)
  CTX.sections_finder = sf

  worksheet = google_spread.open('Charter test results').worksheet('_contracts_contents_focus')
  
  _row=2  
  # Populate document names
  for filename in sorted(contracts):
    head, tail = os.path.split(filename)  
    worksheet.update_cell( _row, 1, tail)
    _row+=1

  #----------
#   batch_contract_find_sections
  #----------
  

#TAKES TIME=====================================================================
if batch_contract_find_sections:
  run_batch(contract_sections_batch_method, worksheet, contracts_filename_prefix, contracts)
#===============================================================================


In [0]:
CTX.__dict__

# Experiments

In [0]:
raise stop here

###### rendering (copy)

In [0]:
import matplotlib as mpl
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt


# //import
def to_color_text(tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
  if len(tokens) == 0:
    return " - empty -"
  if len(weights) != len(tokens):
    raise ValueError("number of weights differs weights={} tokens={}".format(len(weights), len(tokens)))

  #   if()
  vmin = weights.min()
  vmax = weights.max()

  if _range is not None:
    vmin = _range[0]
    vmax = _range[1]

  if print_debug:
    print(vmin, vmax)

  norm = mpl.colors.Normalize(vmin=vmin - 0.5, vmax=vmax)
  html = ""
  cmap = mpl.cm.get_cmap(colormap)

  for d in range(0, len(weights)):
    word = tokens[d]
    if word == ' ':
      word = '&nbsp;_ '

    html += '<span title="{} {:.4f}" style="background-color:{}">{} </span>'.format(
      d,
      weights[d],
      mpl.colors.to_hex(cmap(norm(weights[d]))),
      word)

    #     html+='<span style="background-color:' +mpl.colors.to_hex(cmap(norm(weights[d]) ))+ '">' + str(tokens[d]) + " </span>"
    if tokens[d] == '\n':
      html += "<br>"

  return html



def render_color_text(tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
  html = to_color_text(tokens, weights, colormap, print_debug, _range)
  display(HTML(html))
      
      

def render_contents(doc):
  html = '<h3>Выявленное Содержание документа</h4>'
  html += "<ul>"
  for i in doc.structure.headline_indexes:
    line = doc.structure.structure[i].to_string(doc.tokens_cc)
    html += f'<li> {line} <sup>line {i}</sup></li>'
  html += "</ul>"

  display(HTML(html))


def render_sections(sections):
  html = '<h3>Выявленное секции документа</h4>'
  html += "<ul>"
  for section_type in sections:
    section:HeadlineMeta = sections[section_type]
    body = section.body.untokenize_cc()[:1000]
    headline = section.subdoc.untokenize_cc()[:500]
    #     line = doc.structure.structure[i].to_string(doc.tokens_cc)
    html += f'<li><h3> {headline} <sup>type: {section_type}</sup> </h3> <p>{body}</p> </li>'
  html += "</ul>"

  display(HTML(html))

## 🐩make_headline_attention_vector 

In [0]:



CTX = GLOBALS__['ContractAnlysingContext']
doc = LegalDocument( contracts['/content/gdrive/My Drive/GazpromOil/Contracts/'+'ДОГОВОР_Детский дом.docx'] )
doc.parse()
CTX.contract = doc
print("1000 TOKENS:", CTX.contract.untokenize()[0:1000])
CTX.contract.embedd(CTX.hadlines_factory)




In [0]:
import numpy as np



v = CTX.sections_finder.make_headline_attention_vector(doc)
headlines_attention_vector = CTX.sections_finder.normalize_headline_attention_vector(v)
render_color_text(doc.tokens, headlines_attention_vector)


In [0]:
doc.calculate_distances_per_pattern(CTX.hadlines_factory, pattern_prefix='headline.', merge=True)
headlines_attention_vector_1 = np.ones(len(doc.tokens))


In [0]:
import matplotlib as mpl
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np



def mixclr(color_map, dictionary, min_color=None):
  
  reds = None
  greens = None
  blues = None
  
  for c in dictionary:
    vector = np.array(dictionary[c])
    if reds is None:
      reds = np.zeros(len(vector))
    if greens is None:
      greens = np.zeros(len(vector))
    if blues is None:
      blues = np.zeros(len(vector))
   

    vector_color = color_map[c]

      
    reds += vector * vector_color[0]
    greens += vector*vector_color[1]
    blues += vector*vector_color[2]

#   reds/=len(color_map) 
#   greens/=len(color_map) 
#   blues/=len(color_map) 
  if min_color is not None:
    reds+=min_color[0]
    greens+=min_color[1]
    blues+=min_color[2]
  def cut_(x):
    up = [ min(i,1) for i in x ]
    down = [ max(i,0) for i in x ]
    return down
  
  return np.array([cut_(reds), cut_(greens), cut_(blues)]).T



def render_multicolor_text(tokens, vectors, colormap, min_color=None):
  colors = mixclr(colormap, vectors, min_color=min_color)
  html=''
  for i in range(len(tokens)):
    c = colors[i]
    r= int(255*c[0])
    g= int(255*c[1])
    b= int(255*c[2])
    if tokens[i]=='\n':
      html+='<br>'
    html+=f'<span style="background:rgb({r},{g},{b});"> {tokens[i]} </span>'
  display(HTML(html))
  
  
  
color_map={
    'subj':(0.5,0.5,1),
    'price':(1,0.5,0.5)
}
    
v =  {
    'subj': [1,0,0.5,0],
    'price':[0,0.5,0.5,0.9]
}

render_multicolor_text('lorem ipsum sid amed'.split(' '),  v, color_map, min_color=(0.2,0.2,0.2))

In [0]:
# factory_replacement = ContractHeadlinesPatternFactory(GLOBALS__['elmo_embedder'])
# CTX.hadlines_factory=factory_replacement
doc.calculate_distances_per_pattern(CTX.hadlines_factory, pattern_prefix='headline.', merge=True)
headlines_attention_vector_1 = np.ones(len(doc.tokens))

In [0]:
from patterns import improve_attention_vector
def _find_charter_section_start(doc, headline_pattern_prefix, headlines_attention_vector, additional_attention):

  assert headlines_attention_vector is not None

  vectors = filter_values_by_key_prefix(doc.distances_per_pattern_dict, headline_pattern_prefix)
  # v = rectifyed_sum(vectors, 0.3)
  v = max_exclusive_pattern(vectors)
  v = relu(v, 0.6)

  if additional_attention is not None:
    additional_attention_s = smooth_safe(additional_attention, 6)
    v += additional_attention_s
  
#   v, _ = improve_attention_vector(doc.embeddings, v, relu_th=0.1)
  v *= (headlines_attention_vector+0.1)
  if max(v)>0.75:
    v, _ = improve_attention_vector(doc.embeddings, v, relu_th=0.0)
  
  doc.distances_per_pattern_dict["ha$."+headline_pattern_prefix]=v
  return  v


# v = _find_charter_section_start(doc, 'headline.subj', headlines_attention_vector, additional_attention=None)
 


mc_ =0.3 
color_map={
    'subj':(0,0.2,1),
    'price':(1,0,0),
    'pricecond':(0,1,0),
    'forcemajor':(1,1,0)
}
from ml_tools import *
vectors={
    'subj': CTX.sections_finder._find_charter_section_start(doc, 'headline.subj', headlines_attention_vector, additional_attention=None)[2],
    'price': CTX.sections_finder._find_charter_section_start(doc, 'headline.price', headlines_attention_vector, additional_attention=None)[2],
    'pricecond': CTX.sections_finder._find_charter_section_start(doc, 'headline.pricecond', headlines_attention_vector, additional_attention=None)[2],
    'pricecond': CTX.sections_finder._find_charter_section_start(doc, 'headline.rights', headlines_attention_vector, additional_attention=None)[2]
}

 
render_multicolor_text(doc.tokens, vectors2, color_map, min_color=(0.2,0.2,0.2))

 

## Price (experiments)


In [0]:
GLOBALS__

In [0]:
DOC="""

.1 . Общая сумма договора составляет 41752,62 рублей ( Сорок одна тысяча семьсот пятьдесят два рубля ) 62 копейки , в том числе НДС ( 18 % ) 6369,05 рублей ( Шесть тысяч триста шестьдесят девять рублей ) 05 копеек , в соответствии с локальными сметными расчетами ( Приложение №№1-3 ) . 
2.2 . Окончательная стоимость работ определяется по фактически выполненным работам , при этом порядок определения договорной цены остается неизменным . Подрядчик является самостоятельным плательщиком налогов и сборов в соответствии с законодательством Российской Федерации .
"""
_init_contracts()

from patterns import AbstractPatternFactoryLowCase
from legal_docs import rectifyed_sum_by_pattern_prefix
from ml_tools import *
class ContractValuePatternFactory(AbstractPatternFactoryLowCase):

  def __init__(self, embedder):
    AbstractPatternFactoryLowCase.__init__(self, embedder)

    self._build_sum_patterns()
    self.embedd()

  def _build_sum_patterns(self):
    def cp(name, tuples):
      return self.create_pattern(name, tuples)

    suffix = '(млн. тыс. миллионов тысяч рублей долларов копеек евро)'
    prefix = 'решений о совершении сделок '
    
      

    cp('_phrase.1', ('общая', 'сумма', 'договора составляет'))
    
    
    cp('_sum.work.1', ('Стоимость Работ составляет', '0 рублей', suffix))
    cp('_sum.work.2', ('Расчеты по договору. Стоимость оказываемых услуг составляет ', '0', suffix))
    cp('_sum.work.3', ('Стоимость расчетов по договору не может превышать', '0', suffix))
    cp('_sum.work.4', ('после выставления счета оплачивает сумму в размере', '0', suffix))
    cp('_sum.work.5', ('Общая сумма договора составляет', '0', suffix))

    cp('sum_neg.phone', ('телефон', '00-00-00', ''))

    cp('sum_neg.penalty', ('уплачивается', 'штраф', '0 рублей а также возмещаются понесенные убытки'))
    cp('sum_neg.3', (
      'В случае нарушения  сроков выполнения Работ по соответствующему Приложению , Заказчик имеет право взыскать пени в размере',
      '0%', 'от стоимости не выполненного вовремя этапа Работ по соответствующему Приложению за каждый день просрочки'))
    cp('sum_neg.date.1', ('в срок не позднее, чем за 0 банковских', 'календарных', ' дней'))
    cp('sum_neg.vat', ('в том числе', 'НДС', '0 ' + suffix))
    cp('sum_neg.date.2', ('в течение', '0', 'рабочих дней '))
    
   
    
    

    
from demo import ContractDocument2
cvpf = ContractValuePatternFactory(GLOBALS__['elmo_embedder'])

ctx = GLOBALS__['ContractAnlysingContext']


doc = ContractDocument2(DOC)
doc.parse()
render_contents(doc)
  
# embedded_headlines = doc.embedd_headlines(ctx.hadlines_factory)
# hl_meta_by_index = doc.match_headline_types(ctx.hadlines_factory.headlines, embedded_headlines, 'headline.', 0.9)
# doc.sections = doc.find_sections_by_headlines(hl_meta_by_index)



# values = ctx.fetch_value_from_contract(doc)

from demo import _try_to_fetch_value_from_section
values = _try_to_fetch_value_from_section(doc, cvpf)  




In [0]:
def make_contract_value_attention_vectors(subdoc):
    sumphrase_attention_vector = max_exclusive_pattern_by_prefix(subdoc.distances_per_pattern_dict, '_phrase')
    sumphrase_attention_vector = momentum(sumphrase_attention_vector,0.99)
                                                                      
    value_attention_vector, _c1 = rectifyed_sum_by_pattern_prefix(subdoc.distances_per_pattern_dict, '_sum.work',
                                                                  relu_th=0.4)
    value_attention_vector = cut_above(value_attention_vector, 1)
    value_attention_vector = relu(value_attention_vector, 0.6)
    value_attention_vector = momentum(value_attention_vector, 0.8)

    novalue_attention_vector = max_exclusive_pattern_by_prefix(subdoc.distances_per_pattern_dict, 'sum_neg')
     

    novalue_attention_vector_local_contrast = relu(novalue_attention_vector, 0.6)
    novalue_attention_vector_local_contrast = momentum(novalue_attention_vector_local_contrast, 0.9)

    value_attention_vector_tuned = (value_attention_vector - novalue_attention_vector * 0.7)


    
    value_attention_vector_tuned = (value_attention_vector_tuned + sumphrase_attention_vector)/2
    value_attention_vector_tuned = relu(value_attention_vector_tuned, 0.6)
     

    return {
      'sumphrase_attention_vector':sumphrase_attention_vector,
      'value_attention_vector': value_attention_vector,
      'novalue_attention_vector': novalue_attention_vector,

      'novalue_attention_vector_local_contrast': novalue_attention_vector_local_contrast,
      'value_attention_vector_tuned': value_attention_vector_tuned,
      
    }
  
  
interresting_vectors = make_contract_value_attention_vectors(doc)
# for k in vectors:


fig = plt.figure(figsize=(20, 7))
ax = plt.axes()
offf=0
for i in interresting_vectors: 
  print(i)
  render_color_text(doc.tokens, interresting_vectors[i], _range=(0,1))
  ax.plot(interresting_vectors[i]+offf, label=i, alpha=0.5);
  offf+=1

plt.legend(loc='upper right')


# import re
# number_re = re.compile(r'\d+[,.]\d+')

# def split_by_number(tokens: List[str], attention: List[float], threshold):
#   # TODO: mind decimals!!

#   indexes = []
#   last_token_is_number = False
#   for i in range(len(tokens)):
   
    
#     if attention[i] > threshold and number_re.findall(tokens[i]) is not None :
#       if not last_token_is_number:
#         indexes.append(i)
#       last_token_is_number = True
#     else:
#       last_token_is_number = False

#   text_fragments = []
#   ranges = []
#   if len(indexes) > 0:
#     for i in range(1, len(indexes)):
#       s = indexes[i - 1]
#       e = indexes[i]
#       text_fragments.append(tokens[s:e])
#       ranges.append((s, e))

#     text_fragments.append(tokens[indexes[-1]:])
#     ranges.append((indexes[-1], len(tokens)))
#   return text_fragments, indexes, ranges


#### detectign
from legal_docs import extract_all_contraints_from_sentence, split_by_number


at_vector = interresting_vectors['value_attention_vector_tuned']
value_section=doc
value_section.distances_per_pattern_dict = {**value_section.distances_per_pattern_dict, **interresting_vectors}





text_fragments, indexes, ranges = split_by_number(value_section.tokens, at_vector, 0.2)

print('text_fragments',text_fragments)
print('indexes',indexes)
print('ranges',ranges)


# values: List[ProbableValue] = extract_all_contraints_from_sentence(value_section,
#                                                                        value_section.distances_per_pattern_dict[
#                                                                          'value_attention_vector_tuned'])
  
# for value in values:
#   print(value.confidence, value.value.value)



In [0]:
for value in values:
  print(value.confidence, value.value.value)


## Subjects (experimental)

In [0]:
# raise Ну-ка стой тут!

In [0]:
contracts_filename_prefix='/content/gdrive/My Drive/GazpromOil/Contracts/'
contracts = read_documents(contracts_filename_prefix)


In [0]:


from demo import ContractDocument2
from legal_docs import HeadlineMeta, rectifyed_sum_by_pattern_prefix




 
_init_contracts()
ctx = GLOBALS__['ContractAnlysingContext']




  

  
subj_subdocs=[]

if False:
  for c in contracts:
    #   print (contracts[c][:200])
    print('='*100)
    print(c)
    print('-'*100)
    contract_text = contracts[c]
    doc = ContractDocument2(contract_text)
    doc.parse()

    #====================================
    embedded_headlines = doc.embedd_headlines(ctx.hadlines_factory)
    hl_meta_by_index = doc.match_headline_types(ctx.hadlines_factory.headlines, embedded_headlines, 'headline.', 0.9)
    doc.sections = doc.find_sections_by_headlines(hl_meta_by_index)
    #====================================

    if False:
      render_sections(doc.sections)
      render_contents(doc)

    if 'subj' in doc.sections:
      subj_section=doc.sections['subj']
  #     render_sections( {'subj':subj_section} )

      subj_ = subj_section.body

      #===================
  #     subj_.embedd(ctx.subj_factory)
  #     subj_.calculate_distances_per_pattern(ctx.subj_factory)
  #     r = make_subj_attention_vectors(subj_)
      #===================
  #     render_color_text(subj_.tokens_cc, r['charity_attention_vector'])

      subj_subdocs.append(subj_)
    else:
      print('⚠️ раздел о предмете договора не найден')
    del doc
  
  


In [0]:
import gc
gc.collect()

##### subjs

In [0]:
SUBJS ="""
"Настоящее Дополнительное соглашение является неотъемлемой частью Договора.
Настоящее Дополнительное соглашение вступает в силу с даты его подписания Сторонами и действует до момента полного исполнения Сторонами всех обязательств по Договору. 
 
 Настоящее Дополнительное соглашение составлено в 3-х экземплярах, имеющих равную силу, один экземпляр для Принципала, два экземпляра для Гаранта."




⚠️ раздел о предмете любви не обнаружен




"1. ПРЕДМЕТ ДОГОВОРА.
1.1 Благотворитель оплачивает следующий счет, выставленный на Благополучателя: 
 1.1.1. Счет № 115 на приобретение спортивного оборудования( теннисный стол, рукоход с перекладинами, шведская стенка). Стоимость оборудования 80 000,00( восемьдесят тысяч рублей рублей 00 копеек) рублей, НДС не облагается."




"ДОГОВОР.
о безвозмездной помощи 
( пожертвование) 
 
 г. Санкт-Петербург 
 « 17 » июля 2018 год. 
 Общество с ограниченной ответственностью « Газпромнефть-Региональные продажи », именуемое в дальнейшем « Благотворитель », в лице генерального директора Петрова Семена Анатольевича, действующего на основании Устава, с одной стороны, и Фонд « Олимп », именуемое в дальнейшем « Благополучатель », в лице председателя правления Шарфиков Валерия Львовича, действующего на основании Устава, с другой стороны, именуемые совместно « Стороны », а по отдельности « Сторона », заключили настоящий Договор о нижеследующем: 
 Предмет Договора 
 1.1. « Благотворитель » обязуется передать « Благополучателю » в порядке добровольного пожертвования денежные средства в сумме 1500000( один миллион пятьсот тысяч) рублей 00 копеек для создания и организации работы интернет-платформы « Олимп » по поддержке стартапов в сфере взаимопомощи. 
 Права и обязанности Сторон 
 « Благотворитель » перечисляет денежные средства в сумме "




"Статья 1. ПРЕДМЕТ ДОГОВОРА.
1.1. По условиям Договора Арендодатель обязуется предоставить Арендатору за плату во временное пользование и владение( аренду) недвижимое имущество – Помещение, а Арендатор обязуется принять данные помещения во временное владение и пользование и уплачивать арендную плату в порядке и на условиях, предусмотренных настоящим Договором. 
 1.2. Арендодатель распоряжается Помещением на основании права собственности, о чем выдано свидетельство о государственной регистрации права собственности серия от 20-10-2018 год за регистрационным номером. 
 1.3. Цель использования Арендатором арендуемого Помещения – организация офиса, конторы, представительства. Не допускается использование Арендатором Помещения в иных целях, в том числе в целях осуществления производственной деятельности, для складских целей, в качестве жилого помещения, содержания птиц и животных, хранения вещей, изъятых из гражданского оборота, ядовитых или опасных веществ или предметов, а также иного имущества, хранение которого запре"




"ПРЕДМЕТ ДОГОВОРА.
В соответствии с настоящим договором Залогодержатель принимает, а Залогодатель передает в залог в качестве обеспечения исполнения обязательств по опционному договору № 262/Д от 01-12-2018 год, заключенному в городе Москве между Покупателем Общество с ограниченной ответственностью « Газпромнефть-Региональные продажи »( Должником) и Залогодержателем( Поставщиком)( далее –договор опциона), недвижимое имущество, указанное в пункте 1.2. настоящего Договора. 
 Общий размер обеспечиваемого Предметом залога обязательства составляет 20 000 000( двадцать миллионов) рублей( залоговая стоимость в том числе НДС по ставке, установленной законодательством РФ). 
 Поставщик обязуется обеспечить наличие в резервуарах на объектах Покупателя собственных нефтепродуктов и предоставляет Покупателю опцион на их приобретение с целью дальнейшей реализации, а Покупатель обязуется обеспечить ответственное хранение нефтепродуктов, принять их в свою собственность и оплатить, либо вернуть Поставщику. Опцион действуе"




"Предмет Договора.
1.1. Арендодатель обязуется предоставить Арендатору во временное пользование Помещение, находящееся в здании по адресу: г. Нижний Новгород, улица Тельмана, дом 20, комната №5. Арендатор обязуется принять Помещение, выплачивать Арендодателю арендную плату, а также выполнять другие обязательства, предусмотренные настоящим договором. 
 1.2. Объект аренды находится на первом этаже здания и в соответствии с поэтажным планом включает в себя помещение №5 площадью 17,1 кв. м.( далее « Помещение »). Общая площадь Помещения составляет 17,1 кв. м. Экспликация( фрагмент из поэтажного плана Комплекса) с границами Помещения является Приложением № 1 к настоящему договору. 
 Стороны подтверждают, что указанные данные позволяют определенно установить Помещение, подлежащее передаче Арендатору в качестве объекта аренды. 
 1.3. Право Арендодателя на предоставляемое в аренду Помещение подтверждается Свидетельством о государственной регистрации права. 
 1.4. Помещение передается Арендатору в целях организац"









"ПРЕДМЕТ ДОГОВОРА.
1.1 Продавец обязуется передать в собственность, а Покупатель обязуется принять и оплатить в соответствии с условиями Договора следующее имущество: объекты, земельные участки, движимое имущество: 
 1.1.1. Нежилое здание( АЗС), назначение: нежилое здание, площадь 11,4 кв. м., находящееся по адресу: Российская Федерация, Новосибирская область, село Половинное."




"Статья 1. ПРЕДМЕТ
Заказчик поручает, а Исполнитель обязуется оказывать следующие Услуги/Работы: 
 Услуги по уборке внутренних помещений; 
 Услуги по предоставлению грязезащитных ковров, их чистке и замене на основании заявок или иных письменных указаний Заказчика. Периодичность и стоимость замены грязезащитных ковров указаны в Приложении №2 к Договору. 
 Месторасположение и площадь Объекта, перечень оказываемых Услуг/Работ, объем и график оказания Услуг/Работ по уборке указаны в Приложении № 1."




"5. ПРИЛОЖЕНИЯ К ДОГОВОРУ.
5.1. Приложение №1- Акт приёма благотворительного пожертвования. 
 5.2. Приложение №2 – Спецификация."




"СРОК ДЕЙСТВИЯ, ПОРЯДОК ИЗМЕНЕНИЯ И РАСТОРЖЕНИЯ ДОГОВОРА.
8.1. Договор вступает в силу с момента его подписания обеими сторонами и действует до полного исполнения сторонами своих обязательств по нему, а в части осуществления расчетов до полного их завершения. 
 8.2. Изменение или досрочное расторжение Договора осуществляется по соглашению сторон."




"1. Предмет договора
1.1. Исполнитель предоставляет, а Заказчик оплачивает обучение лиц, определяемых в направлении Заказчика( далее Потребитель) по программе: « Водитель по перевозке опасных грузов »: « Базовый курс подготовки », « Специализированный курс по перевозке ОГ в цистернах ». 
 1.2. Срок обучения в соответствии с учебным планом( индивидуальным графиком) составляет: 
 « Базовый курс подготовки »- 28 часа; 
 « Специализированный курс по перевозке ОГ в цистернах »- 16 часов. 
 1.3. Форма обучения – очная. 
 1.4. Образовательные услуги оказываются на территории ГАУ ДПО СО « Армавирский », расположенного по адресу: Свердловская область, г. Арамиль,."




"1. Предмет договора
Заказчик поручает и оплачивает, а Исполнитель предоставляет услуги в виде проведения обучения работников Заказчика( далее « Слушатели ») по утвержденным Исполнителем программам обучения. 
 Содержание, продолжительность и сроки проведения обучения определяются Программами обучения и расписанием учебных занятий Исполнителя. 
 Услуги оказываются непосредственно работникам Заказчика, именуемым далее « Слушатели », которые проходят обучение на основании поданных Заказчиком заявок, оформленных по утвержденной Исполнителем форме."




"1. Предмет договора
1.1. Исполнитель обязуется оказать, а Заказчик принять и оплатить услуги по шиномонтажу колес( далее- услуги) принадлежащих Заказчику автомобилей( далее- ТС), указанных в Приложении №1, а также другие услуги в пределах списка видов услуг, которые содержатся в Прейскуранте цен( Приложение №2). Услуги оказываются по адресу: г. Челябинск, улица Рылеева,20/2. 
 1.2.1. Сумма договора не должна превышать 500 000( пятьсот тысяч) рублей. 
 1.2.2. Отчетным периодом по настоящему договору является календарный месяц."




"1.Предмет договора
1.1.По договору возмездного оказания услуг « Арендодатель » обязуется оказать услуги, указанные в пункт 1.2 настоящего договора, а « Арендатор » обязуется принять и оплатить эти услуги. 
 1.2. « Арендодатель » обязуется оказать следующие услуги: 
 предоставить в аренду плавательную дорожку в бассейне для оздоровительных занятий « Арендатора », именуемые в дальнейшем « Услуги ». 
 1.3. По настоящему договору « Арендатору » предоставляются дорожка по следующему расписанию: 
 Вторник 13-00 
 Четверг 13-00 
 Воскресенье 13-00 
 Воскресенье 18-00 
 
 1.4. На основании подписанного договора указанные в пункте 1.3. дорожки и время резервируются для « Арендатора » на срок не более 3-х( трех) рабочих дней. В случае, если « Арендатор » в указанный срок не перечислил предоплату, зарезервированное время и дорожки снимаются, а заказ аннулируется."




"Предмет договора
1.1. « Исполнитель » обязуется на основании Заявки « Заказчика » оказывать услуги технической помощи по транспортировке с помощью эвакуатора вверенных ему « Заказчиком » автомобилей с грузом или без груза( далее – автотранспорт) в пункт назначения, указанный « Заказчиком » а « Заказчик » обязуется оплатить оказанные « Исполнителем » услуги. 
 1.2. « Исполнитель » осуществляет транспортировку автотранспорта собственными силами. Маршруты, по которым должен быть доставлен автотранспорт, указывается в Заявках « Заказчика »."




"1. ПРЕДМЕТ ДОГОВОРА.
1.1. Настоящий Договор устанавливает общие условия выполнения комплекса работ по разработке рекламного игрового мобильного приложения для корпоративных клиентов, включая разработку креативной концепции игрового мобильного приложения, прототипов низкой и высокой детализации формирование контента, программирование и тестирование, публикацию приложения, авторский надзор и техническую поддержку, для нужд Общество с ограниченной ответственностью « Газпромнефть-Корпоративные продажи », именуемый в дальнейшем « Работы », в порядке и на условиях, предусмотренных в Приложениях к настоящему Договору, которые будут выполняться в случае согласования Сторонами Заявок на выполнение Работ, предоставляемые Исполнителем Заказчику в течение срока действия настоящего Договора. 
 В результате выполнения Работ Исполнитель передает Заказчику готовые к использованию видео произведения и макеты в форматах wmv/wma, mpg, jpeg, png, ai, html для соответствующих произведений, программные продукты, соответствующие"




"ПРЕДМЕТ ДОГОВОРА И ОСНОВАНИЯ ЕГО ЗАКЛЮЧЕНИЯ.
По настоящему Договору Арендодатель обязуется предоставить Арендатору за плату во временное владение и пользование недвижимое имущество – нежилое помещение, указанное в пункт 1.2 Договора, а Арендатор обязуется принять данное нежилое помещение во временное владение и пользование и уплачивать арендную плату в порядке и на условиях, предусмотренных настоящим Договором. 
 Объектом Аренды, передаваемым Арендатору во временное владение и пользование, является нежилое помещение, расположенное в здании, находящемся по адресу: г. Санкт-Петербург, улица Правды, дом 8, оф. 15, кадастровый номер 11:2222:3333333:45( далее – « Здание ») на земельном участке с кадастровым номером 11:2222:3333333. Арендодатель передаёт Арендатору нежилое помещение, находящееся в вышеуказанном Здании, а именно: 
- Помещение площадью 50( пятьдесят) квадратных метров, расположенное на 3( третьем) этаже Здания 
( далее – « Помещение »). 
 Арендодатель гарантирует, что на момент заключения настоящего Договора, сдаваемое в"




"ПРЕДМЕТ ДОГОВОРА.
Заказчик поручает, а Исполнитель обязуется своими силами выполнить работы по разработке вэб-приложения для продукта « Платформа SmartSystem » согласно плану-графику( Приложение 1 к Договору) и Техническому описанию( Приложение 2 к Договору). 
 Исполнитель обязуется приступить к выполнению работ « 21 » мая 2018 года. 
 Работы должны быть выполнены в полном объеме к « 31 » июля 2018 года. 
 Договор действует до полного исполнения Сторонами своих обязательств, возникших из Договора. 
 Отношения, возникающие по настоящему Договору, не являются трудовыми и регулируются исключительно гражданским законодательством. Исполнитель не включается в штат работников Заказчика, правила внутреннего трудового распорядка, утвержденные Заказчиком, на Исполнителя не распространяются. Исполнитель не получает прав, льгот и компенсаций, предусмотренных действующим трудовым законодательством для работников, выполняющих свою работу по трудовому договору( контракту). Заказчик не обязан осуществлять страхование И"

"""

### parsing hardcoded samples

In [0]:
from legal_docs import LegalDocument

SUBJS_ARR=SUBJS.split('\n\n')

tf.logging.set_verbosity('FATAL')


subj_subdocs=[]
for s in SUBJS_ARR[15:18]:
  print("="*100)
  print(s)
  subj_=LegalDocument(s)
  subj_.parse()
  subj_.embedd(ctx.subj_factory)
  subj_.calculate_distances_per_pattern(ctx.subj_factory)
  subj_subdocs.append(subj_)
#   r = make_subj_attention_vectors(subj_)
#   #===================
#   render_color_text(subj_.tokens_cc, r['charity_attention_vector'], _range=(0,2))

In [0]:
from ml_tools import *
import numpy as np


 


subject_types={
    'charity':'благ-ость'.upper(),
    'comm':'грязный коммерс'.upper(),    
    'comm_estate':'недвижуха'.upper(),
    'comm_service':'наказание услуг'.upper() 
}

subject_types_dict={**subject_types, **{'unknown':'предмет не ясен'}}


# ////////IMPORT
def max_exclusive_pattern_by_prefix(distances_per_pattern_dict, prefix ):

  sum = None

  for p in distances_per_pattern_dict:
    if p.startswith(prefix):
      x = distances_per_pattern_dict[p]
      
      if sum is None:
        sum = np.zeros(len(x))

      sum = np.maximum(sum, x)


  return sum


def make_subj_attention_vectors(subdoc, subj_types_prefixes):
  r={}
  for subj_types_prefix in subj_types_prefixes:
    attention_vector = max_exclusive_pattern_by_prefix(subdoc.distances_per_pattern_dict, subj_types_prefix)  
    r[ subj_types_prefix+'attention_vector']=attention_vector
    if len(attention_vector>0):
      attention_vector_l = relu(attention_vector,0.6)
      r[ subj_types_prefix+'attention_vector_l']=attention_vector_l
    else:
      r[ subj_types_prefix+'attention_vector_l']=[]
    
  return r
#   charity_attention_vector = max_exclusive_pattern_by_prefix(subdoc.distances_per_pattern_dict, 't_charity_')  
#   comm_attention_vector = max_exclusive_pattern_by_prefix(subdoc.distances_per_pattern_dict, 't_comm_' )
  
  
#   comm_attention_vector=momentum(comm_attention_vector, 0.9)

#   comm_attention_vector_l = relu(comm_attention_vector,0.6)
#   charity_attention_vector_l = relu(charity_attention_vector,0.6)
  
  
#   return{
#       'charity_attention_vector':charity_attention_vector,
#       'comm_attention_vector':comm_attention_vector,
#       'comm_attention_vector_l':comm_attention_vector_l,
#       'charity_attention_vector_l':charity_attention_vector_l
#   }

for subj_ in subj_subdocs:
  
  prefixes = [f't_{st}_' for st in subject_types ]
  r = make_subj_attention_vectors(subj_, prefixes)
  
  interresting_vectors = [ r[f't_{st}_attention_vector_l'] for  st in subject_types]
  
  interresting_vectors_means = [np.nanmean(x) for x in interresting_vectors]
  interresting_vectors_maxes = [np.nanmax(x) for x in interresting_vectors]
  winner_id = np.argmax(interresting_vectors_means)
  
  winner_t=prefixes[winner_id][2:-1]
  
  
  fig = plt.figure(figsize=(20, 4))
  ax = plt.axes()
  for i in range(len(interresting_vectors)):    
    ax.plot(interresting_vectors[i], label=prefixes[i]);

  plt.legend(loc='upper left')
    
  #===================
  
  score = interresting_vectors_maxes[winner_id]
  if score<0.3:
    winner_t = 'unknown'
  print(winner_t, interresting_vectors_means[winner_id], score, '*'*80)
#   char_m = np.nanmean(r['t_charity_attention_vector_l'])
#   comm_m = np.nanmean(r['t_comm_attention_vector_l'])
  print(subject_types_dict[winner_t])
  if winner_t is not 'unknown':
    render_color_text(subj_.tokens_cc, r[f't_{winner_t}_attention_vector_l'], _range=(0,1))


 

In [0]:
subj_

In [0]:
raise Не уверен, не ходи

### batch

In [0]:
start_from_row=2 #@param {type:"integer"}
worksheet = google_spread.open('Charter test results').worksheet('Contracts')

def subj_batch_method(filename, r, col) -> int:

  text = contracts[filename]
  doc = ContractDocument2(text)
  doc.parse()
  
  embedded_headlines = doc.embedd_headlines(ctx.hadlines_factory)
  hl_meta_by_index = doc.match_headline_types(ctx.hadlines_factory.headlines, embedded_headlines, 'headline.', 0.9)
  doc.sections = doc.find_sections_by_headlines(hl_meta_by_index)
  
  render_sections(doc.sections)

#   doc, values = GLOBALS__['ContractAnlysingContext'].analyze_contract(text)

  if 'subj' in doc.sections:
    section = doc.sections['subj']
    
    render_sections( {'subj':section} )
    
    body = section.body.untokenize_cc()[:1000]
    headline = section.subdoc.untokenize_cc()[:500]
    worksheet.update_cell(r, col, headline+"\n" +body)
  else:
    worksheet.update_cell(r, col, '⚠️ раздел о предмете любви не обнаружен')
#     render_sections( {'subj':subj_section} )
  
 
  
#   if len(values)>0:
#     cnt=1
#     for c in values:      
#       worksheet.update_cell(r+2, col + cnt+1, c.currency)
#       worksheet.update_cell(r+1, col + cnt, c.sign)       
#       worksheet.update_cell(r+1, col + cnt+1, c.value)       
      
#       cnt+=2
#   else:
#     worksheet.update_cell(r, col, "ничегошеньки не найдено!")   
  
  return r + 3
  
  
  
run_batch(subj_batch_method, worksheet, contracts_filename_prefix)

#Garbage

In [0]:
SAMPLE="""
 

Общие положения





 


Уставный капитал


xxvii Уставный капитал Общества составляется из номинальной стоимости долей его Участников и составляет  6 734 244 615 (Шесть миллиардов семьсот тридцать четыре миллиона двести сорок четыре тысячи шестьсот пятнадцать) рублей.



xxxv Общее собрание участников не вправе принимать решение о распределении прибыли Общества между Участниками Общества:

       • до полной оплаты всего уставного капитала Общества;

     

Органы управления


lxxxiv Органами управления Общества являются:


       • Общее собрание участников Общества;


       • Совет директоров – Коллегиальный орган управления;


       • Правление Общества - Коллегиальный исполнительный орган;


       • Генеральный директор - Единоличный исполнительный орган.


Общее собрание участников Общества


lxxxvi Высшим органом управления Общества является Общее собрание участников Общества. Очередное Общее собрание участников Общества  рассматривает и утверждает результаты деятельности Общества за финансовый год и проводится в период с 1 марта по 30 апреля. Проводимые помимо очередного Общие собрания участников Общества являются внеочередными. Внеочередные Общие собрания участников Общества проводятся в случаях, когда этого требуют интересы Общества.


lxxxvii К исключительной компетенции Общего собрания участников Общества относятся следующие вопросы:

    

       12) установление размера, состава, формы и порядка внесения дополнительных вкладов, определение общей стоимости таких дополнительных вкладов, а также иных условий внесения дополнительных вкладов Участниками Общества;
       13) принятие решения о согласии на совершение или о последующем одобрении крупных сделок, связанных с приобретением, отчуждением или возможностью отчуждения Обществом прямо или косвенно имущества, цена или балансовая стоимость которого составляет 25 (Двадцать пять) и более процентов балансовой стоимости имущества Общества по данным бухгалтерской отчетности на последнюю отчетную дату;
       14) принятие решения о согласии на совершение или о последующем одобрении сделок, в совершении которых имеется заинтересованность, если цена или стоимость имущества, являющегося предметом сделки,  превышает 10 (десять) процентов балансовой процентов  балансовой стоимости имущества Общества по данным бухгалтерской отчетности на последнюю отчетную дату.
       15) принятие решения о реорганизации или ликвидации Общества;
       26) принятие решения о согласии на совершение или о последующем одобрении сделок мены, дарения, иных сделок, предусматривающих безвозмездное отчуждение имущества Общества либо оплату (встречное предоставление) в неденежной форме,  одобрение договоров спонсорского и благотворительного характера, независимо от суммы сделки, за исключением случаев, когда такие сделки требуют получения согласия органов управления Общества как крупные сделки и сделки с заинтересованностью;
       27) решение вопросов, отнесенных к компетенции Совета директоров, в случае, если Совет директоров не избран;
       28) решение иных вопросов, отнесенных в соответствии с действующим законодательством к компетенции Общего собрания участников Общества.



Совет директоров Общества


      1. Совет директоров осуществляет общее руководство деятельностью Общества, а также контроль над исполнением решений Общего собрания участников Общества и иной деятельностью его исполнительного органа.

      2. Совет директоров состоит из 4 (Четырех) человек.

      3. К компетенции Совета директоров относится:


      14) утверждение процедур внутреннего контроля за финансово-хозяйственной деятельностью Общества, в том числе утверждение положения о внутрихозяйственном контроле, документальных проверках и ревизиях;

      15) принятие решения о согласии на совершение или о последующем одобрении сделок, связанных с осуществлением денежных выплат и (или) приобретением, отчуждением и возможностью отчуждения, обременением и возможностью обременения Обществом имущества, в рамках которых цена предложения или стоимость отчуждаемого имущества, определенная на момент совершения сделки, превышает в рамках одной сделки или нескольких взаимосвязанных сделок, заключенных в течение 6 (шести) месяцев, в совокупности 50 000 000 (Пятьдесят миллионов) рублей или эквивалент данной суммы в иной валюте, за исключением случаев, когда такие сделки требуют получения согласия органов управления Общества  как крупные сделки и сделки с заинтересованностью, а также для сделок, одобряемых в соответствии с подпунктами 22)-26) пункта 11.2 настоящего Устава, подпунктами 17) –22), 30) пункта 12.3 настоящего Устава;

      16) принятие решения о начале/урегулировании любых судебных споров и разбирательств, цена иска по которым превышает 50 000 000 (Пятьдесят миллионов) рублей (или эквивалент в другой валюте) в том числе, подписание мирового соглашения, признание иска, отказ от иска, а также совершение любых других процессуальных действий, кроме судебных споров, принятие решения, в отношении которых отнесено к компетенции Общего собрания участников Общества;


Правление Общества


 1. Правление является Коллегиальным исполнительным органом Общества. Правление Общества избирается Общим собранием участников Общества в количестве  2 (Двух) человек – членов Правления Общества.


12. В случае непринятия Правлением решения по одному и тому же вопросу в ходе 2 (Двух) надлежаще созванных заседаний Правления по любым причинам, включая отсутствие кворума и недостаточность количества голосов для принятия решения, вопрос, по которому не было принято решение, выносится на рассмотрение Совета директоров (включается в повестку дня следующего созываемого Совета директоров). Во избежание сомнений, решение не может рассматриваться как непринятое для целей настоящего пункта, в случае, если решение не принято вследствие того, что против него проголосовали или воздержались от голосования все члены Правления.

13. К компетенции Правления Общества относятся:



         • согласование кандидатур на должности руководителей структурных подразделений (за исключением филиалов и представительств) Общества, а также условий трудовых договоров с указанными лицами, согласование освобождения от должностей указанных лиц;


         • принятие решения о согласии на совершение или о последующем одобрении сделок, связанных с осуществлением денежных выплат и (или) приобретением, отчуждением и возможностью отчуждения, обременением и возможностью обременения Обществом имущества, стоимость которого в рамках одной сделки или нескольких взаимосвязанных сделок в совокупности составляет от 1 000 000 (Одного) миллиона рублей до 50 000 000 (Пятьдесят миллионов) рублей или эквивалент данной суммы в иной валюте, за исключением случаев, когда  такие сделки требуют получения согласия органов управления Общества  в соответствии с Законом и Уставом;


         • принятие решения по приемке и оплате Обществом выполненных работ по Агентскому договору на строительство объекта «Южно-Приобский газоперерабатывающий завод (ЮП ГПЗ). Газокомпрессорная станция ЮЛТ Приобского месторождения» №10-875 от 29.09.2010 г., заключенному между Обществом и ООО «Газпромнефть-Хантос».


Генеральный директор Общества


cvi Руководство текущей деятельностью Общества осуществляется единоличным исполнительным органом Общества - Генеральным директором. Генеральный директор избирается Общим собранием участников Общества сроком на 3 (Три) года, если решением Общего собрания участников не предусмотрен иной срок. Генеральный директор может быть переизбран решением Общего собрания участников Общества неограниченное количество раз. Генеральный директор действует на основании Устава Общества и трудового договора, заключаемого с ним Обществом.


       • утверждать внутренние документы, регулирующие текущую (операционную) деятельность Общества, и локальные нормативные акты (за исключением локальных нормативных актов, устанавливающих системы оплаты труда и других документов, утверждение которых отнесено к компетенции иных органов управления Общества);

       • вносить вопросы на рассмотрение Общих собраний участников Общества, заседания Совета директоров;

       • принимать решения по другим вопросам, относящимся к его компетенции.



cxlv Недействительность любого положения настоящего Устава не влечет недействительности остальных его положений. В случае вступления в силу новых нормативных актов, в соответствии с которыми отдельные положения настоящего Устава становятся недействительными, и которые требуют внести изменения в Устав, Участники обязаны принять решения о внесении соответствующих изменений в настоящий Устав.

v Полное фирменное наименование Общества на русском языке:

      Общество с ограниченной ответственностью «Южно-Приобский газоперерабатывающий завод».

      Сокращенное наименование на русском языке:

      ООО «Южно-Приобский ГПЗ».


      Полное фирменное наименование Общества на английском языке: Yuzhno-Priobsky Gaz Processing Plant Limited Liability Company.


      Сокращенное наименование на английском языке:  Yuzhno-Priobsky GPP LLC.


"""

GLOBALS__['CharterAnlysingContext'].analyze_charter(SAMPLE)

In [0]:

from IPython.core.display import display, HTML
doc = GLOBALS__['CharterAnlysingContext'].doc
GLOBALS__['renderer'].render_contents(doc)

 
GLOBALS__['renderer'].render_charter_parsing_results(
    GLOBALS__['CharterAnlysingContext'].org,
    GLOBALS__['CharterAnlysingContext'].constraints)


In [0]:

uploaded = interactive_upload('Charter')  
org, rz = GLOBALS__['CharterAnlysingContext'] .analyze_charter(uploaded[0])




In [0]:
_init_contracts()

In [0]:
GLOBALS__

In [0]:
uploaded = interactive_upload('Договор')  


In [0]:
GLOBALS__['ContractAnlysingContext'].analyze_contract(uploaded[0])

In [0]:


h = GLOBALS__['renderer'].render_constraint_values(GLOBALS__['CharterAnlysingContext'].constraints)
display(HTML(h))


